In [2]:
!pip install opencv-python


   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 435.7 kB/s eta 0:01:29
   ---------------------------------------- 0.1/38.6 MB 744.7 kB/s eta 0:00:52
   ---------------------------------------- 0.2/38.6 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.3/38.6 MB 1.3 MB/s eta 0:00:30
   ---------------------------------------- 0.4/38.6 MB 1.4 MB/s eta 0:00:28
   ---------------------------------------- 0.5/38.6 MB 1.5 MB/s eta 0:00:26
    --------------------------------------- 0.6/38.6 MB 1.6 MB/s eta 0:00:25
    --------------------------------------- 0.6/38.6 MB 1.6 MB/s eta 0:00:24
    --------------------------------------- 0.7/38.6 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.8/38.6 MB 1.6 MB/s eta 0:00:24
    --------------------------------------- 0.9/38.6 MB 1.7 MB/s eta 0:00:23
   - ----

In [1]:
import os
import cv2
import numpy as np
import pandas as pd

# Define paths
images_path = r'C:\Users\Student\Desktop\Shalu\cepha400\cepha400'
output_path = r'C:\Users\Student\Desktop\Shalu\19X400X150_random_test1patch90'
coordinate_output_path = r'C:\Users\Student\Desktop\Shalu\19X400X150_test1patch90_coordinates'
train_csv_path = r'C:\Users\Student\Desktop\Shalu\test1_senior.csv'

# Load the training images and their landmark coordinates
train_df = pd.read_csv(train_csv_path)

# Create output folders if they don't exist
os.makedirs(output_path, exist_ok=True)
os.makedirs(coordinate_output_path, exist_ok=True)

# Define the size of the patches
patch_size = 90
half_patch_size = patch_size // 2

# Loop through each training image
for idx, row in train_df.iterrows():
    image_path = os.path.join(images_path, row['image_path'])
    image_filename = row['image_path'].split('.')[0]
    image = cv2.imread(image_path)

    if image is None:
        print(f"Failed to read image: {image_path}")
        continue

    # Loop through each landmark
    for landmark_index in range(1, 20):  # Assuming there are 19 landmarks
        landmark_x = row[f'{landmark_index}_x']
        landmark_y = row[f'{landmark_index}_y']

        # Create a directory for each landmark
        landmark_dir = os.path.join(output_path, f"landmark_{landmark_index}")
        os.makedirs(landmark_dir, exist_ok=True)

        coordinates_list = []
        for patch_idx in range(400):
            x_min = max(landmark_x - half_patch_size, 0)
            y_min = max(landmark_y - half_patch_size, 0)
            x_max = min(landmark_x + half_patch_size, image.shape[1] - half_patch_size)
            y_max = min(landmark_y + half_patch_size, image.shape[0] - half_patch_size)

            if x_max <= x_min or y_max <= y_min:
                print(f"Skipping patch generation for landmark {landmark_index} at edge.")
                continue

            patch_x = np.random.randint(x_min, x_max - half_patch_size + 1)
            patch_y = np.random.randint(y_min, y_max - half_patch_size + 1)

            patch = image[patch_y:patch_y + patch_size, patch_x:patch_x + patch_size]

            landmark_x_patch = landmark_x - patch_x
            landmark_y_patch = landmark_y - patch_y

            patch_filename = f"{image_filename}_landmark{landmark_index}_patch{patch_idx}.png"
            cv2.imwrite(os.path.join(landmark_dir, patch_filename), patch)
            coordinates_list.append({
                'patch_filename': patch_filename,
                'landmark_index': landmark_index,
                'x': landmark_x_patch,
                'y': landmark_y_patch
            })

        # Construct CSV file path
        csv_file_path = os.path.join(coordinate_output_path, f"landmark_{landmark_index}_coordinates.csv")

        # Check if CSV file already exists
        if os.path.exists(csv_file_path):
            # Load existing data
            existing_data = pd.read_csv(csv_file_path)
            # Append new data
            new_coordinates_df = pd.DataFrame(coordinates_list)
            updated_data = pd.concat([existing_data, new_coordinates_df])
            # Save updated data
            updated_data.to_csv(csv_file_path, index=False)
        else:
            # Save new data if file does not exist
            new_coordinates_df = pd.DataFrame(coordinates_list)
            new_coordinates_df.to_csv(csv_file_path, index=False)

        #print(f"Landmark {landmark_index}: Appended {len(coordinates_list)} coordinates.")
